#Initializations

In [ ]:
!pip install nltk
!pip install autocorrect
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg
import numpy as np
import pickle as pkl
from google.colab import drive


In [ ]:
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/CS772/Assg1/'

#Dataset Creation

In [ ]:
book_names = gutenberg.fileids()
sentences_list = []
for i in range(len(book_names)):
  book_sents = gutenberg.sents(book_names[i])
  sentences_list+= list(book_sents)
print(len(sentences_list))


In [ ]:
from nltk import tokenize
import re
with open(data_path + f'Analogy_dataset.txt', 'r') as f:
  analogy_words = f.read().split()
analogy_words = [*set(analogy_words)]
#print(len(analogy_lines))

num_sent_max = 300

import requests
from bs4 import BeautifulSoup

for i in range(len(analogy_words)):
  wordi = analogy_words[i]
  # Making a GET request
  r = requests.get('https://en.wikipedia.org/wiki/' + wordi)
  # Parsing the HTML
  soup = BeautifulSoup(r.content, 'html.parser')
  #print(soup.prettify())
  ss = soup.find_all('div', class_ ='mw-parser-output')
  counter = 0
  indic = 0
  for s in ss:
    content = s.find_all('p')

    for line in content:
      thisline = line.text
      thissents = tokenize.sent_tokenize(thisline)
      for sentence in thissents: #checking all sentences of a particular wiki page
        if wordi in sentence:
          while not sentence[0].isalpha():  #to get a clean sentence without [21] etc from wikipedia.
            sentence = sentence[1:]
          counter = counter+1
          sentences_list.append(sentence)
          sentence = "".join(re.split("\(|\)|\[|\]", sentence)[::2])
          print(sentence)
          if counter > num_sent_max:
            indic=1
            break
      if indic == 1:
        break

In [ ]:
len(sentences_list)

Data Pre-processing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from autocorrect import Speller

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('omw-1.4')
  

stop_words = set(stopwords.words("english"))
english_words = set(words.words())

spell = Speller()

def preprocess_sentences(sentences):
    preprocessed_sentences = []
    for words in sentences:
        # words = word_tokenize(sentence)
        # words = [spell(word) for word in words]
        words = [word.lower() for word in words]
        words = [word for word in words if word.isalpha() and word not in stop_words and word in english_words]
        words = list(set(words))
        preprocessed_sentence = " ".join(words)
        preprocessed_sentences.append(preprocessed_sentence)
    return list(set(preprocessed_sentences))

# sentences = [
#     "This is a sentence for spell corection.",
#     "Another sentence for spell correctiong",
#     "Stop words are removed in this sentence",
#     "This sentence is lemmatized",
#     "And this sentence is deduplicated"
# ]

# preprocessed_sentences = preprocess_sentences(sentences)
# print(preprocessed_sentences)
processed_sentences = preprocess_sentences(sentences_list)

In [ ]:
with open(data_path + f"processed_sentences.pkl","wb") as f:
  pkl.dump(processed_sentences, f)

#Unimportant/can be deleted

In [ ]:

from tqdm import tqdm

# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# gradient
def sigmoid_derivative(out):
    return out * (1 - out)

# Define the training function for the CBOW model
def train_cbow(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300
    batch_size = 50



    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    for epoch in range(epochs):
        # Loop through each sentence in the training data
        numtotal = 0
        for sentence in tqdm(sentences_list):
            # Initialize the hidden layer activations
            hidden_layer = np.zeros((1, l2_neurons))
            words = sentence.split()
            # Loop through each word in the sentence
            for i, word in enumerate(words):
                numtotal += 1
                # Look up the word vector for the current word
                list_of_onehot_vecs = []
                j = 1
                while(i-j>=0 and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i - j])] = 1
                    list_of_onehot_vecs.append(target_word_vector)
                    j +=1
                j = 1
                while(i +j < len(words) and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i + j])] = 1
                    list_of_onehot_vecs.append(target_word_vector) 
                    j +=1

                word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)
                
                # Compute the input to the hidden layer
                hidden_layer_input = np.dot(word_input_vector, W1)

                # Compute the activations in the hidden layer
                hidden_layer = sigmoid(hidden_layer_input)

                # Compute the input to the output layer
                output_layer_input = np.dot(hidden_layer, W2)

                # Compute the activations in the output layer
                output_layer = sigmoid(output_layer_input)

                # Compute the target word vector
                target_word_vector = np.zeros((1, vocab_size))
                target_word_vector[0, vocab.index(words[i])] = 1
                

                # Compute the error in the output layer
                error = target_word_vector - output_layer

                # Compute the gradients in the output layer
                output_layer_delta = -error * sigmoid_derivative(output_layer)

                # Compute the error in the hidden layer
                hidden_layer_error = np.dot(output_layer_delta, W2.T)

                # Compute the gradients in the hidden layer
                hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)

                # Update the weights
                W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
                W1 -= learning_rate * np.dot(word_input_vector.T, hidden_layer_delta)


    # Return the trained word vectors and weight matrices
    return W1, W2

In [ ]:

from tqdm import tqdm

# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# gradient
def sigmoid_derivative(out):
    return out * (1 - out)

# Define the training function for the CBOW model
def train_cbow(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300
    batch_size = 50



    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    for epoch in range(epochs):
        # Loop through each sentence in the training data
        numtotal = 0
        for sentence in tqdm(sentences_list):
            # Initialize the hidden layer activations
            hidden_layer = np.zeros((1, l2_neurons))
            words = sentence.split()
            # Loop through each word in the sentence
            for i, word in enumerate(words):
                numtotal += 1
                # Look up the word vector for the current word
                list_of_onehot_vecs = []
                j = 1
                while(i-j>=0 and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i - j])] = 1
                    list_of_onehot_vecs.append(target_word_vector)
                    j +=1
                j = 1
                while(i +j < len(words) and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i + j])] = 1
                    list_of_onehot_vecs.append(target_word_vector) 
                    j +=1

                word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)
                
                
                # Compute the input to the hidden layer
                hidden_layer_input = np.dot(word_input_vector, W1)

                if(len(list_of_onehot_vecs)==0):
                  continue
              word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)
              #print("pre_vect",word_input_vector.shape)
              total_batch_vecsin.append(word_input_vector)
              target_word_vector = np.zeros((1, vocab_size))
              target_word_vector[0, vocab.index(words[i])] = 1
              total_batch_vecsout.append(target_word_vector)
              #print("total_batch",np.array(total_batch_vecsin).shape)

          if numtotal % batch_size == 0:
            # Initialize the hidden layer activations
                word_input_vector = np.array(total_batch_vecsin)
                target_word_vector = np.array(total_batch_vecsout)

                print('word_input shape', word_input_vector.shape)

                total_batch_vecsin=[]
                total_batch_vecsout=[]

                print("word_vect",word_input_vector.shape)



                # Compute the activations in the hidden layer
                hidden_layer = sigmoid(hidden_layer_input)

                # Compute the input to the output layer
                output_layer_input = np.dot(hidden_layer, W2)

                # Compute the activations in the output layer
                output_layer = sigmoid(output_layer_input)

                # Compute the target word vector
                target_word_vector = np.zeros((1, vocab_size))
                target_word_vector[0, vocab.index(words[i])] = 1
                

                # Compute the error in the output layer
                error = target_word_vector - output_layer

                # Compute the gradients in the output layer
                output_layer_delta = -error * sigmoid_derivative(output_layer)

                # Compute the error in the hidden layer
                hidden_layer_error = np.dot(output_layer_delta, W2.T)

                # Compute the gradients in the hidden layer
                hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)

                # Update the weights
                W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
                W1 -= learning_rate * np.dot(word_input_vector.T, hidden_layer_delta)


    # Return the trained word vectors and weight matrices
    return W1, W2

In [ ]:
# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# gradient
def sigmoid_derivative(out):
    return out * (1 - out)

# Define the training function for the CBOW model
def train_cbow_batched(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300
    



    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    batch_size = 20
    numtotal = 0

    for epoch in range(epochs):
        total_batch_vecsin=[]
        total_batch_vecsout=[]
        
        # Loop through each sentence in the training data
        for sentence in tqdm(sentences_list):
          numtotal = numtotal+1
          
          hidden_layer = np.zeros((1, l2_neurons))
          words = sentence.split()
          # Loop through each word in the sentence
          list_of_onehot_vecs = []

          for i, word in enumerate(words):
              # Look up the word vector for the current word
              j = 1
              while(i-j>=0 and j<context):
                  target_word_vector = np.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i - j])] = 1
                  list_of_onehot_vecs.append(target_word_vector)
                  j +=1
              j = 1
              while(i +j < len(words) and j<context):
                  target_word_vector = np.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i + j])] = 1
                  list_of_onehot_vecs.append(target_word_vector) 
                  j +=1
              
              if(len(list_of_onehot_vecs)==0):
                  continue
              word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)
              #print("pre_vect",word_input_vector.shape)
              total_batch_vecsin.append(word_input_vector)
              target_word_vector = np.zeros((1, vocab_size))
              target_word_vector[0, vocab.index(words[i])] = 1
              total_batch_vecsout.append(target_word_vector)
              #print("total_batch",np.array(total_batch_vecsin).shape)

          if numtotal % batch_size == 0:
            # Initialize the hidden layer activations
                word_input_vector = np.array(total_batch_vecsin)
                target_word_vector = np.array(total_batch_vecsout)

                #print('word_input shape', word_input_vector.shape)

                total_batch_vecsin=[]
                total_batch_vecsout=[]

                #print("word_vect",word_input_vector.shape)

                # Compute the input to the hidden layer
                hidden_layer_input = np.dot(word_input_vector, W1)

                # Compute the activations in the hidden layer
                hidden_layer = sigmoid(hidden_layer_input)

                # Compute the input to the output layer
                output_layer_input = np.dot(hidden_layer, W2)

                # Compute the activations in the output layer
                output_layer = sigmoid(output_layer_input)

                # Compute the error in the output layer
                error = target_word_vector - output_layer

                # Compute the gradients in the output layer
                output_layer_delta = -error * sigmoid_derivative(output_layer)

                # Compute the error in the hidden layer
                hidden_layer_error = np.dot(output_layer_delta, W2.T)

                # Compute the gradients in the hidden layer
                hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)

                # Update the weights
                out1 = np.zeros((hidden_layer.shape[0], hidden_layer.shape[2], output_layer_delta.shape[2]))
                out2 = np.zeros((hidden_layer.shape[0], word_input_vector.shape[2], hidden_layer_delta.shape[2]))

                def parallelize(hidden_layer, hidden_layer_delta, word_input_vcetor, output_layer_delta):
                    out1 = np.dot(hidden_layer.T, output_layer_delta)
                    out2 = np.dot(word_input_vector.T, hidden_layer_delta)

                print('hidden: ', hidden_layer.shape, 'output delta: ', output_layer_delta.shape, "input: ", word_input_vector.shape, 'hidden: ', hidden_layer_delta.shape)
                #result = functorch.vmap(parallelize)(hidden_layer, hidden_layer_delta, word_input_vector, output_layer_delta)
                
                W2 -= learning_rate * np.dot(hidden_layer.squeeze().T, output_layer_delta.squeeze())
                W1 -= learning_rate * np.dot(word_input_vector.squeeze().T, hidden_layer_delta.squeeze())
                
                # for ind in range(hidden_layer.shape[0]):
                #    out1[ind] = np.dot(hidden_layer[i].T, output_layer_delta[i])
                #    out2[ind] = np.dot(word_input_vector[i].T, hidden_layer_delta[i])
                # out1 = np.einsum('ijk,ijk->ij', hidden_layer, output_layer_delta)
                # out2 = np.einsum('ijk,ijk->ij', word_input_vector, hidden_layer_delta)

                # print("out 1 shape",np.dot(hidden_layer.reshape((hidden_layer.shape[0],hidden_layer.shape[2], hidden_layer.shape[1])), output_layer_delta).shape)
                # print("out 2 shape",np.dot(word_input_vector.reshape((word_input_vector.shape[0],word_input_vector.shape[2], word_input_vector.shape[1])), hidden_layer_delta).shape)

                # W2 -= learning_rate * np.sum(out1, axis = 0)
                # W1 -= learning_rate * np.sum(out2, axis = 0)

        break

    # Return the trained word vectors and weight matrices
    return W1, W2

In [ ]:
#import numpy as np
a = [np.array([0,1,0]), np.array([1,0,0])]
print(np.mean(a, axis = 0))

In [ ]:
W1, W2 = train_cbow_batched(processed_sentences, learning_rate=0.1, epochs=5)
processed_sentences[0]

with open('cbow.npy', 'wb') as f:
    np.save(f, W1)
    np.save(f, W2)

#Important stuff starts here

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import functorch
import pickle as pkl
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/CS772/Assg1/'



# activation function
def softmax(x):
    e_x = torch.exp(x - torch.max(x))
    return e_x / e_x.sum()

# gradient
def softmax_derivative(out, truth):
    indices = torch.argmax(truth, 1, keepdim=True)
    return  -(torch.where(truth>0, out, 0)-(torch.gather(out, 1, indices)*out))*1/(torch.gather(out, 1, indices))

# Define the training function for the CBOW model
def train_cbow_torch(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300

    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = torch.randn(vocab_size, l2_neurons, requires_grad=True)
    W2 = torch.randn(l2_neurons, vocab_size, requires_grad=True)

    batch_size = 100
    numtotal = 0

    for epoch in range(epochs):
        indic_batch = 0
        total_batch_vecsin=[]
        total_batch_vecsout=[]
        
        # Loop through each sentence in the training data
        for inum, sentence in enumerate(tqdm(sentences_list)):
          
          #Saving checkpoints to deal with colab bullshit
          if (inum+1)%(len(sentences_list)/4) == 0:
            with open(data_path + f"W1.pkl","wb") as f:
              a = [inum, epoch, W1]
              pkl.dump(a, f)
            with open(data_path + f"W2.pkl","wb") as f:
              a = [inum, epoch, W2]
              pkl.dump(a, f)
          
          hidden_layer = torch.zeros((1, l2_neurons))
          words = sentence.split()
          # Loop through each word in the sentence
          list_of_onehot_vecs = []

          for i, word in enumerate(words):
              numtotal = numtotal+1
              if ((numtotal+1) % batch_size) == 0:
                indic_batch = 1
              # Look up the word vector for the current word
              j = 1
              while(i-j>=0 and j<context):
                  target_word_vector = torch.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i - j])] = 1
                  list_of_onehot_vecs.append(target_word_vector)
                  j +=1
              j = 1
              while(i +j < len(words) and j<context):
                  target_word_vector = torch.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i + j])] = 1
                  list_of_onehot_vecs.append(target_word_vector) 
                  j +=1
              
              if(len(list_of_onehot_vecs)==0):
                  continue
              word_input_vector = torch.mean(torch.cat(list_of_onehot_vecs, dim=0), dim=0).unsqueeze(0)
              total_batch_vecsin.append(word_input_vector)
              target_word_vector = torch.zeros((1, vocab_size))
              target_word_vector[0, vocab.index(words[i])] = 1
              total_batch_vecsout.append(target_word_vector)
              #print(target_word_vector.shape)

          if indic_batch == 1:
              # Initialize the hidden layer activations
              if (numtotal+1)//batch_size % 20 == 0:
                print(" Batch number : ", (numtotal+1)//batch_size, " of Epoch number : ", epoch) 

              word_input_vector = torch.cat(total_batch_vecsin, dim=0)
              target_word_vector = torch.cat(total_batch_vecsout, dim=0)
              
              print("clean", inum)
              total_batch_vecsin=[]
              total_batch_vecsout=[]

              # Compute the input to the hidden layer
              hidden_layer = torch.matmul(word_input_vector, W1)

              # Compute the input to the output layer
              output_layer_input = torch.matmul(hidden_layer, W2)

              # Compute the activations in the output layer
              output_layer = softmax(output_layer_input)

              # Compute the gradients in the output layer
              output_layer_delta = softmax_derivative(output_layer, target_word_vector)   #this fn includes both dL/dO and dO/dNet

              # Compute the error in the hidden layer
              hidden_layer_error = torch.matmul(output_layer_delta, W2.t())

              W2 = W2 - learning_rate * torch.matmul(torch.squeeze(hidden_layer).t(), torch.squeeze(output_layer_delta))
              W1 = W1 - learning_rate * torch.matmul(torch.squeeze(word_input_vector).t(), torch.squeeze(hidden_layer_error))

              indic_batch = 0

        break

    # Return the trained word vectors and weight matrices
    return W1, W2


Mounted at /content/drive


In [ ]:
with open(data_path + f"processed_sentences.pkl","rb") as f:
  processed_sentences = pkl.load(f)

W1, W2 = train_cbow_torch(processed_sentences, learning_rate=0.1, epochs=5)
processed_sentences[0]

with open('cbow.npy', 'wb') as f:
    np.save(f, W1)
    np.save(f, W2)

['shadrach', 'diary', 'hamstring', 'sentimentalism', 'palm', 'overspread', 'potluck', 'uprightly', 'owing', 'dozen', 'institute', 'moorish', 'forgot', 'sainted', 'striven', 'needlework', 'alternation', 'mica', 'bases', 'refute', 'disagreed', 'triform', 'consign', 'situated', 'gilded', 'fitness', 'unintelligence', 'persuasive', 'whistlingly', 'greedy', 'growing', 'munition', 'tragically', 'awfulness', 'lombard', 'relinquish', 'common', 'sinking', 'railer', 'inclusive', 'arcadian', 'selfsame', 'check', 'clad', 'sluice', 'valvular', 'bracket', 'jane', 'trigonometry', 'inhospitable', 'guillotine', 'profluent', 'enough', 'monologue', 'unsubduedness', 'harbinge', 'save', 'gang', 'folded', 'notch', 'whereto', 'begin', 'oftest', 'add', 'untottering', 'agnostically', 'hosanna', 'simultaneousness', 'distribute', 'offspring', 'vainly', 'trimming', 'added', 'enfold', 'miching', 'empty', 'unspared', 'casually', 'steaming', 'replacement', 'sucking', 'brown', 'satisfactory', 'aroma', 'ninety', 'hungr

  0%|          | 11/92385 [00:00<14:27, 106.43it/s]

clean 20


  0%|          | 22/92385 [00:00<1:06:04, 23.30it/s]

clean 31


...

clean 1801


  2%|▏         | 1802/92385 [01:16<3:31:51,  7.13it/s]

clean 1813


  2%|▏         | 1814/92385 [01:17<2:21:47, 10.65it/s]

clean 1832


In [ ]:
 with open('cbow.npy', 'rb') as f:
    W1 = np.load(f)
    W2 = np.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The vocabulary size is-

In [ ]:
len(word_vec)

Analogy Operation

In [ ]:
def word_vector(word):
  vec = np.zeros((1, vocab_size))
  vec[0, vocab.index(word)] = 1
  return vec


def valid_analogy_CBOW(W1,W2):

  filename = "Analogy_dataset.txt"

  with open(filename, "r") as file:

    corr = 0
    tot = 0

    for line in file:
        
      [x,y,a,b] = line.split()

      x_vec = word_vector(x)
      y_vec = word_vector(y)
      a_vec = word_vector(a)
      b_vec = word_vector(b)

      input_vec = x_vec + b_vec - a_vec 

      hidden_layer_input = np.dot(input_vec, W1)
      # Compute the activations in the hidden layer
      hidden_layer = sigmoid(hidden_layer_input)
      # Compute the input to the output layer
      output_layer_input = np.dot(hidden_layer, W2)
      # Compute the activations in the output layer
      output_layer = sigmoid(output_layer_input)

      max_value = max(output_layer)
      y_pred = output_layer.index(max_value)

      if y_pred == y:
        corr+=1
      tot+=1

      acc_analogy = corr/tot

      return acc_analogy


In [ ]:
##def validate_analogy_skipgram(W1,W2):


In [ ]:
# Skip-gram
def train_skipgram(sentences_list, learning_rate=0.1, epochs=1000):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300

    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    for epoch in range(epochs):
        # Loop through each sentence in the training data
        for sentence in tqdm(sentences_list):

            # Initialize the hidden layer activations
            hidden_layer = np.zeros((1, l2_neurons))
            words = sentence.split()
            # Loop through each word in the sentence
            for i, word in enumerate(words):
                # Look up the word vector for the current word
                list_of_onehot_vecs = []
                j = 1
                while(i-j>=0 and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i - j])] = 1
                    list_of_onehot_vecs.append(target_word_vector)
                    j +=1
                j = 1
                while(i +j < len(words) and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i + j])] = 1
                    list_of_onehot_vecs.append(target_word_vector) 
                    j +=1

                word_input_vector = np.zeros((1, vocab_size))
                word_input_vector[0, vocab.index(words[i])] = 1
                
                
                for k in range(len(list_of_onehot_vecs)):
                  # Compute the input to the hidden layer
                  hidden_layer_input = np.dot(word_input_vector, W1)
                  hidden_layer = sigmoid(hidden_layer_input)
                  output_layer_input = np.dot(hidden_layer, W2)
                  output_layer = sigmoid(output_layer_input)

                  target_word_vector = list_of_onehot_vecs[k]

                  # Compute the error in the output layer
                  error = target_word_vector - output_layer
                  # Compute the gradients in the output layer
                  output_layer_delta = -error * sigmoid_derivative(output_layer)
                  # Compute the error in the hidden layer
                  hidden_layer_error = np.dot(output_layer_delta, W2.T)
                  # Compute the gradients in the hidden layer
                  hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)
                  # Update the weights
                  W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
                  W1 -= learning_rate * np.dot(word_input_vector.T, hidden_layer_delta)


    # Return the trained word vectors and weight matrices
    return W1, W2


In [ ]:
W1, W2 = train_skipgram(processed_sentences, learning_rate=0.1, epochs=5)
processed_sentences[0]
